# Introduction to Linear Regression
Copied and adapted from OpenStats Intro ["Introduction to Linear Regression" lab](http://htmlpreview.github.io/?https://github.com/andrewpbray/oiLabs-base-R/blob/master/simple_regression/simple_regression.html), a product of OpenIntro that is released under a [Creative Commons Attribution-ShareAlike 3.0 Unported](http://creativecommons.org/licenses/by-sa/3.0). Original lab was adapted for OpenIntro by Andrew Bray and Mine Çetinkaya-Rundel from a lab written by the faculty and TAs of UCLA Statistics.

See the [original](http://htmlpreview.github.io/?https://github.com/andrewpbray/oiLabs-base-R/blob/master/simple_regression/simple_regression.html) for more context and explanation. The [data set](https://www.openintro.org/stat/data/?data=mlb11) contains data from 30 Major League Baseball teams in the 2011 season.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot 

In [ ]:
# This cell replicates a function used in the original lab. Run it, but don't dwell on the contents.
%matplotlib nbagg
class plot_ss:
    """ plot_ss(x, y)
        Interactive scatter plot of x,y data. User defines fit line.
    """
    def __init__(self, x, y, showSquares=False):
        self.datax = x
        self.datay = y
        self.showsq = showSquares
        
        self.linex = []
        self.liney = []
        
        # y = m x + b
        self.m = None
        self.b = None
        #self.m = 0.77 
        #self.b = 5032 
        
        
        self.fig = plt.figure()
        self.g = self.fig.add_subplot(111)
        self.g.scatter(x, y)
        self.xlim = self.g.get_xlim()
        self.ylim = self.g.get_ylim()
        self.g.set_title("Click two points to make a line")
        self.fig.canvas.mpl_connect('button_press_event', self.on_press)
        #self.residual()

    def on_press(self, event):
        if len(self.linex) < 2:
            x, y = event.xdata, event.ydata
            self.g.plot(x, y, 'r*', scalex=False, scaley=False)
            self.linex.append(x)
            self.liney.append(y)
           
            if len(self.linex)==2:
                xvals = np.linspace(*self.g.get_xlim())
                yvals = self.fit_line(xvals)
                self.g.plot(xvals, yvals, "r", scalex=False, scaley=False)
                res2 = self.residual()
                self.g.set_title("line: y = {:n} x + {:n}; sum of squares = {:n}".format(self.m, self.b, res2))
                
        return 

    def fit_value(self, x):
        if self.m is None:
            self.m = (self.liney[1] - self.liney[0])/(self.linex[1] - self.linex[0])
            self.b = (self.liney[0]*self.linex[1] - self.linex[0]*self.liney[1])/(self.linex[1] - self.linex[0])
            print(self.m, self.b)
        return self.m * x + self.b
    
    def fit_line(self, xvals):
        print("fit_line")
        return [self.fit_value(x) for x in xvals]        
    
    def residual(self):
        res2 = 0
        for x,y in zip(self.datax, self.datay):
            yhat = self.fit_value(x)
            res2 += pow(y-yhat, 2)
            self.g.vlines(x, min(y, yhat), max(y, yhat), linestyle="--", color="c")
            if self.showsq:
                self.squares(x, y, yhat)
        self.g.set_xlim(*self.xlim)
        self.g.set_ylim(*self.ylim)
        return res2
    
    def squares(self, x, y, yhat):
        res = y - yhat
        x1 = x - res
        self.g.vlines(x1, min(y, yhat), max(y, yhat), linestyle=":", color="r")
        self.g.hlines(y, min(x, x1), max(x, x1), linestyle=":", color="r")
        self.g.hlines(yhat, min(x, x1), max(x, x1), linestyle=":", color="r")
        return
        

## The data

In [ ]:
# load data into dataframe
mlb11 = pd.read_csv("https://www.openintro.org/stat/data/mlb11.csv")

### Exercise 0
Examine the data frame, checking for number of rows and columns, variable names and types, etc.

In addition to runs scored, there are seven traditionally used variables in the data set: at-bats, hits, home runs, batting average, strikeouts, stolen bases, and wins. There are also three newer variables: on-base percentage, slugging percentage, and on-base plus slugging. For the first portion of the analysis we’ll consider the seven traditional variables. At the end of the lab, you’ll work with the newer variables on your own.

### Exercise 1
What type of plot would you use to display the relationship between `runs` and one of the other numerical variables? Plot this relationship using the variable at_bats as the predictor. Does the relationship look linear? If you knew a team’s `at_bats`, would you be comfortable using a linear model to predict the number of runs?

If the relationship looks linear, we can quantify the strength of the relationship with the correlation coefficient.

In [ ]:
mlb11.runs.corr(mlb11.at_bats)

## Sum of squared residuals
Think back to the way that we described the distribution of a single variable. Recall that we discussed characteristics such as center, spread, and shape. It’s also useful to be able to describe the relationship of two numerical variables, such as `runs` and `at_bats` above.

### Exercise 2
Looking at your plot from the previous exercise, describe the relationship between these two variables. Make sure to discuss the form, direction, and strength of the relationship as well as any unusual observations.

Just as we used the mean and standard deviation to summarize a single variable, we can summarize the relationship between these two variables by finding the line that best follows their association. Use the following interactive function to select the line that you think does the best job of going through the cloud of points.

After running this command, you’ll be prompted to click two points on the plot to define a line. Once you’ve done that, the line you specified will be shown in black and the residuals in blue. Note that there are 30 residuals, one for each of the 30 observations. Recall that the residuals are the difference between the observed values and the values predicted by the line:

$$ e_i = y_i - \hat{y_i} $$

In [ ]:
plot_ss(mlb11.at_bats, mlb11.runs)

In [ ]:
plot_ss(mlb11.at_bats, mlb11.runs)

In [ ]:
plot_ss(mlb11.at_bats, mlb11.runs)

The most common way to do linear regression is to select the line that minimizes the sum of squared residuals. To visualize the squared residuals, you can rerun the plot command and add the argument `showSquares = True`.

In [ ]:
plot_ss(mlb11.at_bats, mlb11.runs, showSquares=True)

Note that the output from the plot_ss function provides you with the slope and intercept of your line as well as the sum of squares.

### Exercise 3
Using `plot_ss`, choose a line that does a good job of minimizing the sum of squares. Run the function several times. What was the smallest sum of squares that you got? How does it compare to your neighbors?

## The linear model
It is rather cumbersome to try to get the correct least squares line, i.e. the line that minimizes the sum of squared residuals, through trial and error. Instead we can use the `sm.OLS` (ordinary least squares) function from statsmodels library to fit the linear model (a.k.a. regression line).

In [ ]:
# Fit a linear model using statsmodels (sm) OLS method. 
model = sm.OLS(mlb11.runs, sm.add_constant(mlb11.at_bats))

The function `sm.OLS` takes the parameters (response_variable, explanatory_variable). If the line is not forced to have a zero-intercept, a constant must be added to the explanatory variable data using the function `sm.add_constant`. The statement above fits a linear model of `mlb11.runs` as a function of `mlb11.at_bats`. 

The output of the function is an object of the class OLS. The next cell will create a fit and display the summary.

In [ ]:
results = model.fit()
print(results.summary())

Let’s consider the pieces of information in this output important to the task at hand. The first line in the left column shows the name dependent (response) variable, we can verify that this is the model we intended. The table in the middle is key; its first column displays the inear model's y-intercept and the coeffients of `at_bats`. With this table, we can write down the least squares regression line for the linear model:

 $$ \hat{y} = −2789.2429 + 0.6305 ∗ atbats $$

One last piece of information we will discuss from the summary output is the R-squared, or more simply, $R^2$. The $R^2$ value represents the proportion of variability in the response variable that is explained by the explanatory variable. For this model, 37.3% of the variability in runs is explained by at-bats.



### Exercise 4
Fit a new model that uses `homeruns` to predict `runs`. Using the estimates from the output, write the equation of the regression line. What does the slope tell us in the context of the relationship between success of a team and its home runs?

## Prediction and prediction errors
Let's create a scatter plot with the least squared line laid on top.

In [ ]:
fig = plt.figure()
g = fig.add_subplot(111)
g.plot(mlb11.at_bats, results.fittedvalues, 'r')
g.scatter(mlb11.at_bats, mlb11.runs)
g.set_xlabel("at_bats")
g.set_ylabel("runs")

The model results object `results` has an attribute `fittedvalues` that is a pandas Series instance with values $\hat{y}$ for each value of the explanatory variable used to train the model. The line these values describe can be used to predict $y$ at any value of $x$. When predictions are made for values of $x$ that are beyond the range of the observed data, it is referred to as extrapolation and is not usually recommended. However, predictions made within the range of the data are more reliable. They’re also used to compute the residuals.

### Exercise 5
If a team manager saw the least squares regression line and not the actual data, how many runs would he or she predict for a team with 5,578 at-bats? Is this an overestimate or an underestimate, and by how much? In other words, what is the residual for this prediction?

## Model diagnostics
To assess whether the linear model is reliable, we need to check for (1) linearity, (2) nearly normal residuals, and (3) constant variability.

*Linearity*: You already checked if the relationship between runs and at-bats is linear using a scatterplot. We should also verify this condition with a plot of the residuals vs. at-bats. 

In [ ]:
fig = plt.figure()
g = fig.add_subplot(111)
g.scatter(mlb11.at_bats, results.resid)
# add a horizontal dashed line at y=0
g.hlines(0, *g.get_xlim(), linestyles="--")

### Exercise 6
Is there any apparent pattern in the residuals plot? What does this indicate about the linearity of the relationship between runs and at-bats?

*Nearly normal residuals*: To check this condition, we can look at a histogram...

In [ ]:
fig = plt.figure()
g = fig.add_subplot(111)
g.hist(results.resid, bins = np.arange(-150, 225, 50))

...or a normal probability plot of the residuals.

In [ ]:
fig = qqplot(results.resid, line='s') #qqplot displays and returns the figure object

### Exercise 7
Based on the histogram and the normal probability plot, does the nearly normal residuals condition appear to be met?

*Constant variability*:

### Exercise 8 
Based on the plot of residuals at the beginning of this section, does the constant variability condition appear to be met?

## On Your Own
1) Choose another traditional variable from `mlb11` that you think might be a good predictor of runs. Produce a scatterplot of the two variables and fit a linear model. At a glance, does there seem to be a linear relationship?

2) How does this relationship compare to the relationship between `runs` and `at_bats`? Use the $R^2$ values from the two model summaries to compare. Does your variable seem to predict `runs` better than `at_bats`? How can you tell?

3) Now that you can summarize the linear relationship between two variables, investigate the relationships between runs and each of the other five traditional variables. Which variable best predicts `runs`? Support your conclusion using the graphical and numerical methods we’ve discussed (for the sake of conciseness, only include output for the best variable, not all five).

4) Now examine the three newer variables. These are the statistics used by the author of Moneyball to predict a teams success. In general, are they more or less effective at predicting runs that the old variables? Explain using appropriate graphical and numerical evidence. Of all ten variables we’ve analyzed, which seems to be the best predictor of runs? Using the limited (or not so limited) information you know about these baseball statistics, does your result make sense?

5) Check the model diagnostics for the regression model with the variable you decided was the best predictor for runs.